In [1]:
import numpy as np
from numba import vectorize, cuda, jit
from struct import unpack
import cupy as cp
import pandas as pd

In [2]:
class ReadBinHdr(object):

    ''' Read binary trace header for a trace '''

    def __init__(self, bh):
        if len(bh) != 400:
            print("Binary header should be 400 bytes long")
        else:
            self.jobid = unpack(">i", bh[0:4])[0]
            self.line = unpack(">i", bh[4:8])[0]
            self.reel = unpack(">i", bh[8:12])[0]
            self.numtrcens = unpack(">h", bh[12:14])[0]
            self.numauxtrcs = unpack(">h", bh[14:16])[0]
            self.sampint = unpack(">h", bh[16:18])[0]
            self.sampint2 = unpack(">h", bh[18:20])[0]
            self.samppertrc = unpack(">h", bh[20:22])[0]
            self.samppertrc2 = unpack(">h", bh[22:24])[0]
            self.datasampcode = unpack(">h", bh[24:26])[0]
            self.ensfold = unpack(">h", bh[26:28])[0]
            self.sortcode = unpack(">h", bh[28:30])[0]
            self.dis_units = unpack(">h", bh[54:56])[0]
            self.segyformat = unpack(">h", bh[300:302])[0] // 256
            self.lengthflag = unpack(">h", bh[302:304])[0]
            self.numexthdrs = unpack(">h", bh[304:306])[0]

    def __str__(self):
        return "Job ID : {} \nLine Number : {} \nReel Number : {}\
                 \nNumber of traces per ensemble : {} \nNumber of Aux Traces : {}\
                 \nSample interval : {}\nField Sample Interval : {}\nSample per trace :{}\
                 \nField Samples per Trace : {} \nSample Format : {}\nEnsemble Fold : {}\
                 \nTrace Sorting Code : {}\nMeasurement units(1-Meters 2-Feet) : {}\
                 \nSEGY Format : {}\nLength Flag(0-Variable 1-Fixed) : {}\
                 \nNumber of Extended Text : {}".format(self.jobid, self.line, self.reel, \
                 self.numtrcens, self.numauxtrcs, self.sampint, self.sampint2, self.samppertrc, \
                 self.samppertrc2, self.datasampcode, self.ensfold, self.sortcode, self.dis_units, \
                 self.segyformat, self.lengthflag, self.numexthdrs)
    

ibm2ieee = cp.RawKernel(r'''
extern "C" __global__
void ibm2ieee(const unsigned int* x1, float* y) {
    int tid = blockDim.x * blockIdx.x + threadIdx.x;
    unsigned int x = x1[tid];
    if (x != 0){
        int sign = ((x1[tid] >> 31) & 0x01) * (-2) + 1;
        int exponent = (x1[tid] >> 24) & 0x7F;
        int tmp = 4 * (exponent - 64);
        double p;
        if (tmp < 0) {
            int po2 = 1 << (abs(tmp));
            p = (double)(1.0/po2);
        }
        else{
            p = 1 << tmp;
        }
        int mantissa = x1[tid] & 0x00ffffff;
        float frac = ((float)mantissa / 0x1000000);
        y[tid] = sign * frac * p;
    }
    else{
        y[tid] = 0.0;
    }    
}
''', 'ibm2ieee')

#reguar python
def ibmpython(data):
    if data == 0:
        return 0.0
    sign = data >> 31 & 0x01
    exponent = data >> 24 & 0x7f
    mantissa = (data & 0x00ffffff) / float(pow(2, 24))
    return (1 - 2 * sign) * mantissa * pow(16.0, exponent - 64)
#numpy version
def ibm32numpy(data):
    sign = np.bitwise_and(np.right_shift(data,31),1) * (-2) +1
    exponent = np.bitwise_and(np.right_shift(data,24), int('0x7F',0))
    p = np.power(16.0, exponent - 64)
    mantissa = np.bitwise_and(data, int('0x00ffffff', 0))
    frac = mantissa / int('0x1000000', 0)
    return sign * frac * p

In [11]:
def readsegy(file_loc):
    trcs = []
    with open(file_loc,"rb") as f:
        EBCDIC = f.read(3200)
        BIN = ReadBinHdr(f.read(400))
        samples_per_trace = BIN.samppertrc
        sample_interval = BIN.sampint
        idx=1
        print("Starting loop....")
        while True:
            tmp = f.read(240)
            if not tmp:
                print("End scanning .......")
                break
            trc = f.read(samples_per_trace*4)
            trcs.append(trc)
            idx+=1
            if idx % 10000 == 0:
                print("reading trace {}".format(idx))
    return trcs

In [12]:
trcs_kerry = readsegy("../data/Kerry3D.segy")

Starting loop....
reading trace 10000
reading trace 20000
reading trace 30000
reading trace 40000
reading trace 50000
reading trace 60000
reading trace 70000
reading trace 80000
reading trace 90000
reading trace 100000
reading trace 110000
reading trace 120000
End scanning .......


In [13]:
trcs_4t = readsegy("../data/4T_Rapids.segy")

Starting loop....
reading trace 10000
End scanning .......


In [14]:
data_kerry = trcs_kerry[10000]
data_kerry[:400]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xbf\x94\xb9_\xc0\x1b\xe2\xc2\xc0J\\\xb0\xc0f?r\xc0x\xd6\x9d\xc0\x8bm\xc9\xc0\x94\xb9_\xc0\x94\xb9_\xc0S\xa8F@\x8bm\xc9A\x1d\xa0\xeeA%.XA\x12\x97,\xc0.y\xee\xc0\x1b\xe2\xc2@\x82"3\x00\x00\x00\x00\xc1\x10DF\xc1\x18fj\xc0\xcc~\xe3\xc0%.X\xc1\x16\x13\x84\xc1\x1e\xcaa@f?rA\x1a\xb9O@x\xd6\x9d\xc0\x8bm\xc9\xc0\x9e\x04\xf5@\xe8a\xa4A-\xe54A\x18\xfb#\xc1\x10\xd9\x00\xc1\x12\x97,@J\\\xb0@7\xc5\x84\xc1\x16\x13\x84\xc0\\\xf3\xdcA*\xfd\x96\xc1\x17\xd1\xb0\xc1I\xc7\xf6A\x18fjA&\xec\x84@\xa7P\x8bA\x17<\xf7\xc1*h\xdc\xc1\x1e5\xa7A\x1a\xb9O\xc0\xa7P\x8b@\xd5\xcayA-\xe54\xc0\xfa\xf8\xd0\xc14\xdd\xe5\xc0A\x11\x1aA \x88\x8d@o\x8b\x07@7\xc5\x84@\xe8a\xa4\xc1\x18\xfb#\xc1\x1d\x0c5A\x14UXA\x12\x97,\x00\x00\x00\x00@f?r\xc0\x9e\x04\xf5\xc1\x14\xea\x11@\x8bm\xc9A\x10DF\xc0\xb9\xe7\xb7\xc07\xc5\x84@\xc33M\xc0x\xd6\x9d\xc0\xb0\x9c!@\x94\xb9_@\x94\xb9_\xc0\\\xf3\xdc\xc0\\\xf3\xdc?\x94\xb9_?\x94\xb9_\xc0\x94\xb9_\xc0f

In [15]:
data_4t = trcs_4t[0]
data_4t[:400]

b"\xbf@\xb6#\xbf@t\xc9\xbf@\x0c\xad\xbfB\xed5\xbfFf\xa3\xbfE\xc5\x86\xbfD\x86w\xbfD\xabc\xbfEFT\xbfF6\x8b\xbfFS\x0e\xbfE\xeeB\xbfE^\xf4\xbfD<\xae\xbfC\xa7h\xbfD\x01\x19\xbfD\xc6L\xbfE\xebl\xbfF\xb9\x15\xbfF&Z\xbfE2\xee\xbfF\x11\xaf\xbfGJu\xbfF\xea\xcb\xbfF<\x86\xbfF]\x19\xbfG\x9f\x86\xbfIH\xf0\xbfJ|y\xbfK\x11o\xbfJ\xd3\xc5\xbfJcS\xbfJ\x0f\xe6\xbfI\x91\x98\xbfIt\xd5\xbfIw\x19\xbfH\xb3\xa8\xbfHC\x04\xbfId-\xbfJN\xf8\xbfI-C\xbfE+Y\xbf8\xd3\xc5\xbf4\x0b\x0f?c\xca\r?\xf7=^\xc0\x16\xb5\x10\xbf\xcc\xd6\x1f\xc0+\x00\x10\xc1\x1by\xb0\xc17\xb6*\xc18\xcb\x14\xc1+t@\xc1%\xd6[\xc1 \x9f*\xc1\x14U\xf6\xc0\xb1\xca!\xc0!\x9a\x1c@w \xb2@\x86z\xb1@Y )@9\xa0\xe7@\x185\x0b@'\x87l@K\x92\x0c@_10@p\x1d\x0f@t\xcfj@li,@d\xafD@\x82\xc5\x8e@\x9f\xf5\t@\x8f\x15\xfe@\x81\x12\xe3@q\x82c@m\xb40@\x90f\x85@\xa7\xe2\x85@\xa6*)@\x9d\xefR@\x8f\xe8X@\x83I\xea@\x87\x8e\xe3@\x95h\x8c@\x9d\xca\xed@\x9f\x92\x89@\x99o\x8a@\x90$\x8f@\x8d\xce\xb5@\x90\xa9-@\x92\xae\x00@\x93Vi@\x92\xda @\x91\x0f\xf4@\x8e\x11\xde@\x8a}z@\x87\xa0\xc

### Now to run some benchmarks using two different types of data

In [18]:
# vanilla python applied to list comprehension
%timeit np.array([ibmpython(x) for x in np.frombuffer(data_kerry,dtype='>u4')])

7.86 ms ± 9.19 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
# vanilla python applied to list comprehension
%timeit np.array([ibmpython(x) for x in np.frombuffer(data_4t,dtype='>u4')])

63.8 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%timeit cp.array([ibmpython(x) for x in np.frombuffer(data_kerry,dtype='>u4')])

7.9 ms ± 53.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%timeit cp.array([ibmpython(x) for x in np.frombuffer(data_4t,dtype='>u4')])

63.5 ms ± 401 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### THE time for the 4T data is much larger because it has more data and less zeros

In [23]:
%timeit np.array([ibm32numpy(x) for x in np.frombuffer(data_kerry,dtype='>u4')])

21 ms ± 49 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit np.array([ibm32numpy(x) for x in np.frombuffer(data_4t,dtype='>u4')])

69.2 ms ± 213 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%timeit cp.array([ibm32numpy(x) for x in np.frombuffer(data_kerry,dtype='>u4')])

21.1 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit cp.array([ibm32numpy(x) for x in np.frombuffer(data_4t,dtype='>u4')])

69.2 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### it seems the numpy function is not very optimized at all. the addition of a cupy array also had little effect

### lets try vectorizing the python function

In [29]:
%timeit np.vectorize(ibmpython)(np.frombuffer(data_kerry,dtype='>u4'))

429 µs ± 4.84 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
%timeit np.vectorize(ibmpython)(np.frombuffer(data_4t,dtype='>u4'))

2.91 ms ± 5.55 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### This is a amazing result! both data sets experience a 20x uplift in time with only the semantic change to using numpy's vectorize. 
### but lets see if we can still do better by using a cupy kernel

In [32]:
x1 = cp.array(np.frombuffer(data_kerry,dtype='>u4'),dtype=cp.uint32)
arr = cp.zeros(x1.size, dtype=cp.float32)
%timeit ibm2ieee((x1.size,), (1,), (x1, arr))

4.33 µs ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [34]:
x2 = cp.array(np.frombuffer(data_4t,dtype='>u4'),dtype=cp.uint32)
arr2 = cp.zeros(x2.size, dtype=cp.float32)
%timeit ibm2ieee((x2.size,), (1,), (x2, arr2))

12.4 µs ± 637 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Wow this is even faster and represents up to a 200x speedup for the larger dataset. This should not really come as a surprise as we are getting closer to c perfromance.
### however writing Cuda Kernels can be a challenging and represents a bit of a learning curve for the non CUDA programmer. Lets see if Numba can help us here

In [39]:
@vectorize(['float32(uint32)'])
def ibmpython_numba(data):
    if data == 0:
        return 0.0
    sign = data >> 31 & 0x01
    exponent = data >> 24 & 0x7f
    mantissa = (data & 0x00ffffff) / float(pow(2, 24))
    return (1 - 2 * sign) * mantissa * pow(16.0, exponent - 64)

In [42]:
%timeit ibmpython_numba(np.frombuffer(data_kerry,dtype='>u4'))

4.54 µs ± 16.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [44]:
%timeit ibmpython_numba(np.frombuffer(data_4t,dtype='>u4'))

18.3 µs ± 24 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
